In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re

Corpus: The corpus here is an entire book and it is received as a large string.

In [2]:
r = requests.get("https://data.heatonresearch.com/data/t81-558/text/"\
                 "treasure_island.txt")

# raw_test is essentially a large string.
raw_text = r.text

Here we take lower case to every word in the text and then eliminte the non-ASCII characters.

In [3]:
processed_text = raw_text.lower()
processed_tex = re.sub(r'[^\x00-\x7f]',r'', processed_text)

Here we create dictionaries to assign id's to every character in the text.

In [4]:
print('corpus length: ', len(processed_text))
chars = list(set(processed_text))
char_idx = {c: i for i, c in enumerate(chars)}
idx_char = {i: c for i, c in enumerate(chars)}
print('total chars: ', len(chars))

corpus length:  397419
total chars:  67


They define sequences of characters of length max_len from the text taking steps of size step.

In [5]:
max_len = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - max_len, step):
    sentences.append(processed_text[i: i + max_len])
    next_chars.append(processed_text[i + max_len])
    
for i in range(0, 4):
    print('sentence: ', sentences[i])
    print('next_chars: ', next_chars[i])

sentence:  ï»¿the project gutenberg ebook of treasu
next_chars:  r
sentence:  the project gutenberg ebook of treasure 
next_chars:  i
sentence:   project gutenberg ebook of treasure isl
next_chars:  a
sentence:  oject gutenberg ebook of treasure island
next_chars:  ,


Vectorization:

The first dimension of x represents which sequence we are looking at, the second the position in the sequence and the third is which char is the said position and sequence. Then, $x[s, p, c]=1$ if and only if the character with index $c$ is present in the position $p$ of the sequence number $s$.

In the same spirit, $y[s,c]=1$ if and only if the char $c$ is the next char of the sequence $s$.

In [6]:
x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_idx[char]] = 1
    y[i, char_idx[next_chars[i]]] = 1
print(x.shape, y.shape)

(132460, 40, 67) (132460, 67)


Build the Deep Learning model.

In [7]:
model = Sequential()
# First layer: lstm.
model.add(LSTM(128, input_shape=(max_len, len(chars))))
# Second layer: dense with ouput dimension len(chars).
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               100352    
_________________________________________________________________
dense (Dense)                (None, 67)                8643      
Total params: 108,995
Trainable params: 108,995
Non-trainable params: 0
_________________________________________________________________


The next function takes the output preds from the neural network and then, using its values, assign probabilities to each value in the array. Then, sample from a multinomial random variable with this probabilities.

In [9]:
def sample(preds, temperature=1.0):
    """
    Given the array of preds, assign probabilities to each
    value. Then, with this probabilities samples from a multinomial
    random variable.
    """
    preds = np.array(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
def on_epoch_end(epoch, _):
    """
    """
    print("******************************************************")
    print('----- Generating text after Epoch: %d' % epoch)
    start_index = random.randint(0, len(processed_text) \
                                 - max_len -1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        # takes a random sentence from the text.
        sentence = processed_text[start_index: start_index + max_len]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
    
        # completes 400 characters of the random subsequence of the text.
        for i in range(400):
            # create a x-data of size one.
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_idx[char]] = 1.
            # predict the output.
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            # get the char with the index given by the NN.
            next_char = idx_char[next_index]
            # add the generated char to the text.
            generated += next_char
            # delete first char and add the added char.
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
    
    

In [ ]:
# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
1035/1035 [==============================] - 47s 44ms/step - loss: 2.4550
******************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "ould any be intended. he turned and spok"
ould any be intended. he turned and spoke the stread the shore the streed the still the shop to the said the stread the doctor the strome the shore the strear the shope the shore the shoped the stroog the streed the strome the strook the more the shore the said the strought and the strest the shore the start to the street the shop the shop the strome the strome the strook the shop the stread
and the shout the strome the stread the shop
----- temperature: 0.5
----- Generating with seed: "ould any be intended. he turned and spok"
ould any be intended. he turned and spoke in or said
steed and the coom and the hore
come here there sadest in the bock, and the coars out the good the said
connor soon and the shep to the said t